In [77]:
!pip install contractions

In [78]:
import nltk
import spacy
import pickle
import pandas as pd
import contractions
nltk.download("words")
from textblob import TextBlob
from nltk.probability import FreqDist
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [79]:
pd.set_option('display.max_colwidth', None)

In [80]:
spacy_lib = spacy.load("en_core_web_sm")

In [81]:
pd.set_option("display.max_columns", 10)

In [82]:
dataframe = pd.read_json("../input/google-review-erode-hospital/google_reviews_2022-10-21_erode_hospital.json", lines=True)

In [83]:
dataframe.columns

Index(['input', 'user', 'rating', 'review_date', 'formatted_review_date',
       'review_text', 'hospital_name', 'address', 'images',
       'review_availability', 'url', 'extraction_date'],
      dtype='object')

In [84]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2698 entries, 0 to 2697
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   input                  2698 non-null   object
 1   user                   2698 non-null   object
 2   rating                 2698 non-null   object
 3   review_date            2698 non-null   object
 4   formatted_review_date  2698 non-null   object
 5   review_text            2698 non-null   object
 6   hospital_name          2698 non-null   object
 7   address                2698 non-null   object
 8   images                 2698 non-null   object
 9   review_availability    2698 non-null   bool  
 10  url                    2698 non-null   object
 11  extraction_date        2698 non-null   object
dtypes: bool(1), object(11)
memory usage: 234.6+ KB


In [85]:
df_dropped = dataframe.drop(["user", "review_date", "images", "extraction_date", "review_availability", "address", "url", "input"], axis=1)

In [86]:
df_dropped.columns

Index(['rating', 'formatted_review_date', 'review_text', 'hospital_name'], dtype='object')

In [87]:
df_filter = df_dropped.copy()

In [88]:
# df_filter.dropna(subset=["review_text"],inplace=True)

In [89]:
df_filter = df_filter[df_filter.review_text != ""]

In [90]:
df_filter["rating"] = df_dropped["rating"].str.replace(" out of 5,","").str.replace("Rated ", "")

In [91]:
df_filter.rating = df_filter.rating.astype(float)

In [92]:
df_filter["rating"].value_counts()

5.0    1082
1.0     263
4.0     260
3.0      99
2.0      38
Name: rating, dtype: int64

In [93]:
df_filter["bi_ratings"] = df_filter["rating"].apply(lambda x: "Positive" if x>3.0  else "Negative")
# df_filter.loc[df_filter.rating > 3.0, "bi_ratings"] = "Postive"
# df_filter.loc[df_filter.rating <= 3.0, "bi_ratings"] = "Negative"

In [94]:
df_filter

,rating,formatted_review_date,review_text,hospital_name,bi_ratings
0,4.0,2022-09-21 00:00:00,"Dr.mei sir treatment well and good ,hospital is clean... I'm satisfied.... All staffs kind to patient's",Be Well Hospitals Erode,Positive
1,5.0,2022-09-30 00:00:00,"Good treatment, Best consultant pediatric doctor for child.. Doctor advise for parents is too good to increase food habits",Be Well Hospitals Erode,Positive
2,4.0,2022-09-21 00:00:00,Paediatric Dr.Nazir is well experienced and never recommend too many medicines for the baby. Hospital has very good ambience and good hospitality.,Be Well Hospitals Erode,Positive
3,5.0,2022-10-14 00:00:00,They took good care of my mom thank you for recovered my mom very soon thank you so much to each and every person. We thank you from bottom of our heart.,Be Well Hospitals Erode,Positive
4,5.0,2022-10-07 00:00:00,Very nice hospital treatment good uma sistar baby care suppar all good iam really imparased,Be Well Hospitals Erode,Positive
...,...,...,...,...,...
2682,5.0,2021-10-21 00:00:00,Dr. Vanitha is very patient friendly and explains everything clearly. I've been consulting her for ob-gyn advice and she has been nothing short of fantastic.,Sri Jothi Hospital,Positive
2683,4.0,2016-10-21 00:00:00,Good hospital We generally take our kids here for illness. Treatment is good! enough care is given!,Sri Jothi Hospital,Positive
2684,5.0,2020-10-21 00:00:00,I am born at dis hospital.,Sri Jothi Hospital,Positive
2685,5.0,2018-10-21 00:00:00,Great service with humanity,Sri Jothi Hospital,Positive


In [95]:
# df_filter.drop("rating", axis=1, inplace=True)

In [96]:
df_filter.bi_ratings.value_counts()

Positive    1342
Negative     400
Name: bi_ratings, dtype: int64

In [97]:
df_filter = df_filter[df_filter.columns[[3,1,2,0,4]]]

In [98]:
df_filter

,hospital_name,formatted_review_date,review_text,rating,bi_ratings
0,Be Well Hospitals Erode,2022-09-21 00:00:00,"Dr.mei sir treatment well and good ,hospital is clean... I'm satisfied.... All staffs kind to patient's",4.0,Positive
1,Be Well Hospitals Erode,2022-09-30 00:00:00,"Good treatment, Best consultant pediatric doctor for child.. Doctor advise for parents is too good to increase food habits",5.0,Positive
2,Be Well Hospitals Erode,2022-09-21 00:00:00,Paediatric Dr.Nazir is well experienced and never recommend too many medicines for the baby. Hospital has very good ambience and good hospitality.,4.0,Positive
3,Be Well Hospitals Erode,2022-10-14 00:00:00,They took good care of my mom thank you for recovered my mom very soon thank you so much to each and every person. We thank you from bottom of our heart.,5.0,Positive
4,Be Well Hospitals Erode,2022-10-07 00:00:00,Very nice hospital treatment good uma sistar baby care suppar all good iam really imparased,5.0,Positive
...,...,...,...,...,...
2682,Sri Jothi Hospital,2021-10-21 00:00:00,Dr. Vanitha is very patient friendly and explains everything clearly. I've been consulting her for ob-gyn advice and she has been nothing short of fantastic.,5.0,Positive
2683,Sri Jothi Hospital,2016-10-21 00:00:00,Good hospital We generally take our kids here for illness. Treatment is good! enough care is given!,4.0,Positive
2684,Sri Jothi Hospital,2020-10-21 00:00:00,I am born at dis hospital.,5.0,Positive
2685,Sri Jothi Hospital,2018-10-21 00:00:00,Great service with humanity,5.0,Positive


In [99]:
df_filter["contractions_review"] = df_filter["review_text"].apply(lambda x: contractions.fix(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [100]:
# df_filter["spelling_review"] = df_filter["review_text"].apply(lambda x: TextBlob(x).correct())

In [101]:
df_filter.loc[:,["review_text", "contractions_review"]]

,review_text,contractions_review
0,"Dr.mei sir treatment well and good ,hospital is clean... I'm satisfied.... All staffs kind to patient's","Dr.mei sir treatment well and good ,hospital is clean... I am satisfied.... All staffs kind to patient's"
1,"Good treatment, Best consultant pediatric doctor for child.. Doctor advise for parents is too good to increase food habits","Good treatment, Best consultant pediatric doctor for child.. Doctor advise for parents is too good to increase food habits"
2,Paediatric Dr.Nazir is well experienced and never recommend too many medicines for the baby. Hospital has very good ambience and good hospitality.,Paediatric Dr.Nazir is well experienced and never recommend too many medicines for the baby. Hospital has very good ambience and good hospitality.
3,They took good care of my mom thank you for recovered my mom very soon thank you so much to each and every person. We thank you from bottom of our heart.,They took good care of my mom thank you for recovered my mom very soon thank you so much to each and every person. We thank you from bottom of our heart.
4,Very nice hospital treatment good uma sistar baby care suppar all good iam really imparased,Very nice hospital treatment good uma sistar baby care suppar all good iam really imparased
...,...,...
2682,Dr. Vanitha is very patient friendly and explains everything clearly. I've been consulting her for ob-gyn advice and she has been nothing short of fantastic.,Dr. Vanitha is very patient friendly and explains everything clearly. I have been consulting her for ob-gyn advice and she has been nothing short of fantastic.
2683,Good hospital We generally take our kids here for illness. Treatment is good! enough care is given!,Good hospital We generally take our kids here for illness. Treatment is good! enough care is given!
2684,I am born at dis hospital.,I am born at dis hospital.
2685,Great service with humanity,Great service with humanity


In [102]:
words = set(nltk.corpus.words.words())

In [186]:
# df_filter["review_text"] = df_filter["review_text"].apply(lambda x: x.lower())
df_filter["processed_review"] = df_filter["contractions_review"].apply(lambda x: " ".join(word.lower() for word in nltk.wordpunct_tokenize(x) if word.lower() in words))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [180]:
stopword_data = set(["ourselves", "hers", "between", "yourself", "but", "again", "there", "about", "once", "during", "out", "very", "having", "with", "they", "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "most", "itself", "other", "off", "is", "s", "am", "or", "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we", "these", "your", "his", "through", "don", "nor", "me", "were", "her", "more", "himself", "this", "down", "should", "our", "their", "while", "above", "both", "up", "to", "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them", "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then", "that", "because", "what", "over", "why", "so", "can", "did", "now", "under", "he", "you", "herself", "has", "just", "where", "only", "myself", "which", "those", "i", "after", "few", "whom", "t", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it", "how", "further", "was", "here", "than"])

In [187]:
df_filter["processed_review"] = df_filter["processed_review"].apply(lambda x: [token.lemma_ for token in spacy_lib(x) if not token.is_punct and not token.like_num and not token.is_space and not str(token) in stopword_data])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [106]:
# df_filter["processed_review"] = df_filter["processed_review"].apply(lambda x: [token.lemma_ for token in spacy_lib(x) if not token.is_punct and not token.like_num and not token.is_space and not token.is_stop])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [190]:
df_filter["processed_review"] = df_filter["processed_review"].apply(lambda x: [token.lower() for token in x if not len(token.strip())<2 and not len(token.strip())>15])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [191]:
df_filter["processed_review"]

0                                          [sir, treatment, well, good, hospital, clean, satisfied, kind, patient]
1         [good, treatment, good, consultant, pediatric, doctor, child, doctor, advise, too, good, increase, food]
2       [nazir, well, experienced, never, recommend, too, many, baby, hospital, good, ambience, good, hospitality]
3                                [take, good, care, thank, soon, thank, much, every, person, thank, bottom, heart]
4                                                      [nice, hospital, treatment, good, baby, care, good, really]
                                                           ...                                                    
2682                     [patient, friendly, everything, clearly, consult, gyn, advice, nothing, short, fantastic]
2683                               [good, hospital, generally, take, illness, treatment, good, enough, care, give]
2684                                                                            

In [192]:
df_filter["processed_review"] = df_filter["processed_review"].apply(lambda x: TreebankWordDetokenizer().detokenize(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [193]:
df_filter["word_count"] = df_filter["processed_review"].apply(lambda x: [f"{key}-{value}" for key, value in TextBlob(x).word_counts.items() if value > 1])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [195]:
postive_words = set(['absolutely', 'accepted', 'acclaimed', 'accomplished', 'accomplishment', 'admirable', 'amazing', 'authentic', 'abundant', 'abundance', 'achiever', 'affluent', 'astonishing', 'awesome', 'adorable', 'beaming', 'beauty', 'beautiful', 'believe', 'beloved', 'beneficial', 'bestfriend', 'blessed', 'bliss', 'brave', 'breathtaking', 'bright', 'brilliant', 'brilliance', 'bravo', 'calm', 'calming', 'capable', 'captivating', 'caring', 'celebrate', 'certain', 'champ', 'champion', 'charitable', 'charm', 'charming', 'comfortable', 'committed', 'compassion', 'compassionate', 'confident', 'congratulations', 'correct', 'courage', 'courageous', 'courteous', 'creative', 'creativity', 'credible', 'credibility', 'daring', 'darling', 'dazzling', 'dear', 'dearest', 'dedicated', 'delicious', 'delight', 'delightful', 'dependable', 'developed', 'devoted', 'devotion', 'dignified', 'diligent', 'discipline', 'distinguished', 'divine', 'dream', 'dreamy', 'driven', 'dynamite', 'dynamo', 'earnest', 'easy', 'ecstatic', 'education', 'effective', 'effortless', 'elegant', 'empathy', 'eminent', 'empower', 'empowerment', 'endearing', 'enduring', 'energy', 'enjoy', 'enjoyable', 'enormous', 'enthusiasm', 'epic', 'equality', 'especial', 'essential', 'esteem', 'eternal', 'euphoria', 'everlasting', 'excel', 'excellent', 'excellence', 'exceptional', 'exciting', 'excitement', 'exhilarating', 'experienced', 'expert', 'exquisite', 'extensive', 'extraordinary', 'extravagance', 'exuberant', 'fabulous', 'fair', 'faith', 'faithful', 'familiar', 'famous', 'fancy', 'fantastic', 'fascinating', 'favorite', 'fearless', 'fidelity', 'fierce', 'fit', 'first', 'flawless', 'forever', 'forgiveness', 'fortunate', 'fortune', 'free', 'freedom', 'fresh', 'friend', 'fulfilling', 'fun', 'funny', 'generous', 'genius', 'gentle', 'genuine', 'gift', 'gifted', 'giving', 'glam', 'glamour', 'glee', 'glorious', 'goal', 'goodwill', 'gorgeous', 'gourmet', 'grace', 'grateful', 'gratitude', 'great', 'good', 'great', 'greatness', 'growth', 'handsome', 'happy', 'happiness', 'hardworking', 'harmony', 'health', 'healthy', 'heart', 'heaven', 'hello', 'helpful', 'hero', 'hilarious', 'holy', 'home', 'honest', 'honor', 'hope', 'humble', 'huge', 'humane', 'humility', 'humor', 'idyllic', 'imaginative', 'immaculate', 'immense', 'impeccable', 'important', 'impressive', 'incomparable', 'incredible', 'indispensable', 'infinite', 'ingenuity', 'inspiration', 'inspirational', 'inspiring', 'integrity', 'intelligence', 'intrepid', 'irreplaceable', 'irresistible', 'jackpot', 'jewel', 'jolly', 'joy', 'joyful', 'joyous', 'jubilee', 'juicy', 'jumbo', 'just', 'justice', 'keen', 'key', 'kind', 'kindness', 'kindred', 'king', 'kiss', 'knightly', 'knockout', 'knowledge', 'knowledgeable', 'large', 'laugh', 'laughter', 'lavish', 'leader', 'legacy', 'legendary', 'leisure', 'liberty', 'life', 'longevity', 'love', 'loving', 'loyal', 'loyalty', 'luck', 'lucky', 'luxury', 'magic', 'magical', 'magnificent', 'majestic', 'major', 'marvelous', 'massive', 'meaning', 'meaningful', 'merry', 'mighty', 'miracle', 'moral', 'mother', 'motivated', 'natural', 'neat', 'needed', 'necessary', 'new', 'nice', 'noble', 'nourishing', 'nourishment', 'nurse', 'nurture', 'nutritious', 'open', 'openminded', 'opportunity', 'optimism', 'optimistic', 'optimum', 'opulence', 'opulent', 'organic', 'original', 'outstanding', 'overjoy', 'paradise', 'partner', 'passion', 'passionate', 'patience', 'patriotic', 'peace', 'peaceful', 'perfect', 'phenomenal', 'pioneer', 'plenty', 'playful', 'polite', 'positive', 'power', 'powerful', 'precious', 'premium', 'pretty', 'prime', 'principle', 'prize', 'prodigy', 'pride', 'productive', 'promise', 'prosperous', 'proud', 'pure', 'purpose', 'quality', 'qualified', 'queen', 'quiet', 'quick', 'relax', 'rad', 'radiant', 'rapture', 'rainbow', 'ready', 'real', 'reassuring', 'regal', 'rejoice', 'rejuvenate', 'relationship', 'relevant', 'reliable', 'resilient', 'respect', 'respectful', 'responsible', 'restorative', 'rewarding', 'rich', 'righteous', 'romance', 'romantic', 'safe', 'scholar', 'savior', 'scrumptious', 'secure', 'selfless', 'sensational', 'sensible', 'serenity', 'sexy', 'sharp', 'shine', 'significant', 'sincere', 'smile', 'smart', 'sophisticated', 'spark', 'sparkle', 'special', 'splendid', 'star', 'strong', 'stunning', 'success', 'super', 'superb', 'supportive', 'surprise', 'survivor', 'sweet', 'sympathy', 'talent', 'tasty', 'teacher', 'team', 'teamwork', 'tenaciuos', 'thankful', 'thorough', 'thoughtful', 'thrive', 'timeless', 'together', 'togetherness', 'treasure', 'triumph', 'true', 'trust', 'trustworthy', 'truth', 'ultimate', 'unbeatable', 'unbelievable', 'unconditional', 'understanding', 'unforgettable', 'unique', 'united', 'unstoppable', 'uplifting', 'useful', 'utmost', 'utopia', 'unwavering', 'vip', 'valiant', 'valor', 'value', 'victory', 'virtue', 'warm', 'wealth', 'welcome', 'whole', 'wholesome', 'willpower', 'winner', 'wisdom', 'wise', 'wonderful', 'worthy', 'wow', 'xanadu', 'xfactor', 'xoxo', 'yeah', 'yay', 'yes', 'young', 'youth', 'youthful', 'yummy', 'zeal', 'zen', 'zest'])

In [196]:
def postive_negative_word_finder(sentence, token, index):
    try:
        return f"{token} {sentence[index+1]}"
    except:
        return token

In [197]:
df_filter["postive_words"] = df_filter.loc[:,"processed_review"].apply(lambda x: [postive_negative_word_finder(TreebankWordTokenizer().tokenize(x), token, index ) for index, token in enumerate(TreebankWordTokenizer().tokenize(x)) if token in postive_words])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [198]:
negative_words = set(['abrasive', 'apathetic', 'controlling', 'dishonest', 'impatient', 'anxious', 'betrayed', 'disappointed', 'embarrassed', 'jealous', 'abysmal', 'bad', 'callous', 'corrosive', 'damage', 'despicable', 'donot', 'enraged', 'fail', 'gawky', 'haggard', 'hurt', 'icky', 'insane', 'jealous', 'lose', 'malicious', 'naive', 'not', 'objectionable', 'pain', 'questionable', 'reject', 'rude', 'sad', 'sinister', 'stuck', 'tense', 'ugly', 'unsightly', 'vice', 'wary', 'yell', 'zero', 'adverse', 'banal', 'cannot', 'corrupt', 'damaging', 'detrimental', 'dreadful', 'eroding', 'faulty', 'ghastly', 'hard', 'hurtful', 'ignorant', 'insidious', 'junky', 'lousy', 'mean', 'nasty', 'noxious', 'odious', 'perturb', 'quirky', 'renege', 'ruthless', 'savage', 'slimy', 'stupid', 'terrible', 'undermine', 'untoward', 'vicious', 'weary', 'yucky', 'alarming', 'barbed', 'clumsy', 'dastardly', 'dirty', 'dreary', 'evil', 'fear', 'grave', 'ignore', 'injure', 'insipid', 'lumpy', 'menacing', 'naughty', 'none', 'offensive', 'pessimistic', 'quit', 'repellant', 'scare', 'smelly', 'substandard', 'terrifying', 'unfair', 'unwanted', 'vile', 'wicked', 'angry', 'belligerent', 'coarse', 'crazy', 'dead', 'disease', 'feeble', 'greed', 'harmful', 'ill', 'injurious', 'messy', 'negate', 'petty', 'reptilian', 'scary', 'sobbing', 'suspect', 'threatening', 'unfavorable', 'unwelcome', 'villainous', 'woeful', 'annoy', 'bemoan', 'cold', 'creepy', 'decaying', 'disgusting', 'fight', 'grim', 'hate', 'immature', 'misshapen', 'negative', 'nothing', 'oppressive', 'plain', 'repugnant', 'scream', 'sorry', 'suspicious', 'unhappy', 'unwholesome', 'vindictive', 'worthless', 'anxious', 'beneath', 'criminal', 'deformed', 'disheveled', 'filthy', 'grimace', 'hideous', 'imperfect', 'missing', 'never', 'neither', 'poisonous', 'repulsive', 'severe', 'spiteful', 'unhealthy', 'unwieldy', 'wound', 'apathy', 'boring', 'collapse', 'cruel', 'deny', 'dishonest', 'foul', 'gross', 'homely', 'impossible', 'misunderstood', 'no', 'nowhere', 'poor', 'revenge', 'shocking', 'sticky', 'unjust', 'unwise', 'appalling', 'broken', 'confused', 'cry', 'deplorable', 'dishonorable', 'frighten', 'grotesque', 'horrendous', 'inane', 'moan', 'nobody', 'prejudice', 'revolting', 'shoddy', 'stinky', 'unlucky', 'upset', 'atrocious', 'contrary', 'cutting', 'depressed', 'dismal', 'frightful', 'gruesome', 'horrible', 'inelegant', 'moldy', 'nondescript', 'rocky', 'sick', 'stormy', 'unpleasant', 'awful', 'contradictory', 'deprived', 'distress', 'guilty', 'hostile', 'infernal', 'monstrous', 'nonsense', 'rotten', 'sickening', 'stressful', 'unsatisfactory'])

In [199]:
df_filter["negative_words"] = df_filter.loc[:,"processed_review"].apply(lambda x: [postive_negative_word_finder(TreebankWordTokenizer().tokenize(x), token, index ) for index, token in enumerate(TreebankWordTokenizer().tokenize(x)) if token in negative_words])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [200]:
(df_filter.loc[:,["rating", "processed_review", "postive_words", "negative_words"]]).sort_values(by=['rating'])

,rating,processed_review,postive_words,negative_words
2054,1.0,bad not go kind hospital emergency charge like bad,[kind hospital],"[bad not, not go, bad]"
2075,1.0,completely money minded look doctor time register patient registration cost give tablet syrup without doctor consultation money totally disappoint,[],[]
2076,1.0,bad experience consultation specialist not recommend,[],"[bad experience, not recommend]"
2077,1.0,pharmacy service damn slow lethargic kind emergency pathetic must,[kind emergency],[]
654,1.0,bitter experience hospital money mind nicely not action come real problem say insurance coverage doctor go operation even normal delivery possible really not ready think we get mind get kindly check normal delivery percentage strongly not recommend hospital anyone,"[real problem, ready think]","[not action, not ready, not recommend]"
...,...,...,...,...
1030,5.0,mother admit hear uterus problem decide uterus do uterus surgery safely well talented handle care well hospital clean facility available support kindly wee thank care teem thank,[mother admit],[]
1029,5.0,good consultation monthly review pregnancy care take delivery staff baby care sir good care baby,"[good consultation, good care]",[]
1028,5.0,wonderful handling know proper heart give trust worthy,"[wonderful handling, heart give, trust worthy, worthy]",[]
1040,5.0,would like thank madam madam complete care support guidance also look even fine course wife normal delivery actually come erode hospital district keeping mind normal delivery good care stay uncle house near hospital delivery day delivery pain midnight immediately go hospital head nurse team check scan advise we take admission doctor around pain severe delivery room doctor madam also anaesthesia also around baby bear normal delivery madam easily around clock time go hospital discharge take good care handle careful look every finer towards health treatment good hospital come across little expensive worth payable good class treatment cleanly hospital recommend hospital well wish good care good treatment thank support,"[good care, nurse team, team check, good care, health treatment, good hospital, good class, good care, good treatment]","[pain midnight, pain severe, severe delivery]"


In [201]:
# df_filter["textblob_rating"] = df_filter["processed_review"].apply(lambda x: TextBlob(x).sentiment)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [203]:
# (df_filter.loc[df_filter["rating"]==1.0]).loc[:,["rating", "textblob_rating", "processed_review", "postive_words", "negative_words", "word_count"]]

,rating,textblob_rating,processed_review,postive_words,negative_words,word_count
19,1.0,"(0.05625000000000001, 0.5333333333333333)",leave feedback well improvement familiar group hospital visit find system administration not efficient staff roam book hear without clear process flow knowledge disappointed part emergency duty doctor not name take ray blood test day treatment hope management consider healthy way look,"[familiar group, knowledge disappointed, hope management, healthy way]","[not efficient, disappointed part, not name]",[not-2]
100,1.0,"(-0.10951704545454546, 0.48025568181818185)",people friend year old champ new surgical mean safe human life rather strive money busy kill people everything business result poor family please keep distance evil head well without well,"[friend year, champ new, new surgical, safe human, life rather]","[mean safe, poor family, evil head]","[people-2, well-2]"
126,1.0,"(-0.0625, 0.2375)",beware people life people family friend admit due fracture accident surgery tell not finish coz lack technology lose friend never end greed people towards money stay aside human flesh,"[life people, friend admit, friend never]","[not finish, lose friend, never end, greed people]","[people-3, friend-2]"
164,1.0,"(-0.4999999999999999, 0.5833333333333333)",friend common people everything money business humanity bad experience ever stay away hospital,[friend common],[bad experience],[]
175,1.0,"(-0.6999999999999998, 0.6666666666666666)",bad hospital costly,[],[bad hospital],[]
...,...,...,...,...,...,...
2563,1.0,"(0.2, 0.2)",cost too much,[],[],[]
2570,1.0,"(0.0, 0.0)",,[],[],[]
2584,1.0,"(-0.10499999999999994, 0.5166666666666666)",bad treatment hospital matter many time go give send away say see next time permanent cure disease original bad treatment hospital,[original bad],"[bad treatment, disease original, bad treatment]","[bad-2, treatment-2, hospital-2, time-2]"
2585,1.0,"(-0.2479166666666666, 0.5375)",bad reception tell come evening not see token system social gap covid original bad reception evening token system covid,[original bad],"[bad reception, not see, bad reception]","[bad-2, reception-2, evening-2, token-2, system-2, covid-2]"


In [213]:
df_filter.loc[:50,["rating", "word_count", "postive_words", "negative_words"]]

,rating,word_count,postive_words,negative_words
0,4.0,[],"[good hospital, kind patient]",[]
1,5.0,"[good-3, doctor-2]","[good treatment, good consultant, good increase]",[]
2,4.0,[good-2],"[experienced never, good ambience, good hospitality]",[never recommend]
3,5.0,[thank-3],"[good care, heart]",[]
4,5.0,[good-2],"[nice hospital, good baby, good really]",[]
5,5.0,[good-2],"[good doctor, nurse good, good especially]",[]
6,5.0,"[care-2, thank-2]","[excellent care, humble special, special thank]",[]
7,4.0,[],"[good staff, nice hospital]",[]
8,5.0,"[good-4, care-2]","[good service, good treatment, good care, nice sir, good thank]",[]
9,4.0,"[basic-2, available-3, care-3]","[good hospital, health care]",[]
